# CIS 280 Data Visualization and Communication
---

Final Assignment 

### Setting Up All Imports

In [33]:
#Importing for the project
import json
import pathlib

import plotly.express as px
import pandas as pd

# Import the server library
from jupyter_dash import JupyterDash

# Import the standard dash library and the components
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output


In [34]:
#Importing plot utilities
import plot_util

### Initalizing File Load Up

In [78]:
df = pd.read_csv("beer_states_filtered.csv")
df

,Unnamed: 0,state,year,barrels,type
0,0,AK,2008,2067.69,On Premises
1,1,AK,2009,2263.65,On Premises
2,2,AK,2010,1929.15,On Premises
3,3,AK,2011,2251.02,On Premises
4,4,AK,2012,2312.43,On Premises
...,...,...,...,...,...
1848,1848,total,2015,17750504.62,Kegs and Barrels
1849,1849,total,2016,17000382.28,Kegs and Barrels
1850,1850,total,2017,16514866.69,Kegs and Barrels
1851,1851,total,2018,15662381.42,Kegs and Barrels


# Setting Up Interactives

In [79]:
stateradio = dcc.RadioItems(
    id = 'state_radio',
    options=[
    {'label':"Massachusetts", 'value': "MA"},
    {'label':"Rhode Island", 'value': "RI"},
    {'label':"Connecticut", 'value': "CT"}, 
    {'label':"New Hampshire", 'value': "NH"},
    {'label':"New York", 'value': "NY"},
    {'label':"New Jersey", 'value': "NJ"}, 
    {'label':"Pennsylvania", 'value': "PA"},
    {'label':"Delaware", 'value': "DE"},
    {'label':"Maryland", 'value': "MD"}, 
    {'label':"North Carolina", 'value': "NC"},
    {'label':"Virginia", 'value': "VA"},
    {'label':"South Carolina", 'value': "SC"}, 
    {'label':"Georgia", 'value': "GA"} 
    ],
    value = "MA")

In [80]:
typeradio = dcc.RadioItems(
    id = 'type_radio',
    options=[
    {'label':"P", 'value': "On Premises"},
    {'label':"B&C", 'value': "Bottles and Cans"},
    {'label':"K&B", 'value': "Kegs and Barrels"} 
    ],
    value = "On Premises")

In [86]:
wyomingtypeslider = dcc.Slider(
        id='wyoming-type-slider', 
        step=1,
        min=2010,
        max=2019,
        value=2010, 
    ) 

### Dashboard Setup

In [93]:
app = JupyterDash('Beer Distributed Statistics') 

server = app.server

underline = "_________________________________________________________________________________________________________________________________________________________________"

app.layout = html.Div(id="content",children=[ 
    html.H1('Beer Distributed Data'),
    html.P(underline),
    html.H2('Barrels Avaliable Based Off State'),
    dcc.Graph(id='bar_graph',className="leftplots"),
    stateradio,
    html.P(underline),
    html.H2('Barrel Size Based Off Type'),
    dcc.Graph(id='box_plot',className="leftplots"),
    typeradio,
    html.P(underline),
    html.H2('Barrel Size of All Types In The Smallest Population State Wyoming'),
    dcc.Graph(id='violin_plot',className="leftplots"),
    wyomingtypeslider,
    html.P(underline),
    html.H2('Types Used Based Off Beer Distribution From 2018+'),
    dcc.Graph(figure=plot_util.defineScatter(), className="leftplots")
    ])

@app.callback(
    Output('bar_graph', 'figure'),
    Input('state_radio', 'value'))
#Defying callback function
#1st plot
def showBarGraph(value):
    df = pd.read_csv("beer_states_filtered.csv")
    df = df[df['state'] == value]
    fig = px.bar(df,
             x='barrels',
             y='state',
             color='state',
             color_discrete_sequence=px.colors.qualitative.D3,
             title='Barrels Avaliable Based Off State')
    return fig

#2ND PLOT
@app.callback(
    Output('box_plot', 'figure'),
    Input('type_radio', 'value'))
def defineBoxPlot(value):
    df = pd.read_csv("beer_states_filtered.csv")
    df = df[df['type'] == value]
    fig = px.box(df,
             x='barrels',
             y='type',
             color='type',
             color_discrete_sequence=px.colors.qualitative.D3,
             title='Barrel Size Based Off Type')

    return fig

#3rd PLOT
@app.callback(
    Output('violin_plot', 'figure'),
    Input('wyoming-type-slider', 'value'))
def defineViolinPlot(value):
    df = pd.read_csv("beer_states_filtered.csv")
    wyoming = df[df['state']=='WY'].reset_index(drop=True)
    wyoming = wyoming[wyoming['year'] >= 2010].reset_index(drop=True)
    wyoming = wyoming[wyoming['year'] == value]
    fig = px.violin(wyoming,
             y='type',
             x='barrels',
             color='type',
             title='Barrel Size of All Types In The Smallest Population State Wyoming')

    return fig
# Display in browser
app.run_server("jupyterlab")